In [ ]:
#--------Loading in Packages------#
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
from scipy.io import mmread
from scipy.sparse import csr_matrix
import seaborn as sns
import os
import tacco as tc

In [ ]:
#------Loading in Mapped 8um bin filtered dataset-------#
b2c_adata = sc.read_h5ad('/projects/b1217/HHA/EL_S2_B2C/anndatas/B2C_HE_Segmentation_Filtered_Annotated_11_13_25.h5ad')
b2c_adata

In [ ]:
#--------Loading in Full scRNA Atlas-----#
HHA = sc.read_h5ad("/projects/b1217/Edward/R_Projects/HHA/h5ad/20250516_HHA-SC-Final.h5ad")
HHA

In [ ]:
#--------Plotting Full Image-------#
fig, ax = plt.subplots(figsize=(20, 20))
sc.pl.spatial(b2c_adata, color=["FinalAnnotation"], img_key="hires", ax=ax)

In [ ]:
#------Building Melanocyte AnnData------#
ConversionDirectory =  "/projects/b1217/HHA/Multiome_Scanpy_Conversion/Full_Atlas/"
#Reading in the count matrix from seurat.combined
counts = mmread(os.path.join(ConversionDirectory, "HHA_SCRNA_Multiome_Melano_FullConversion_Expression_LogCounts_11_18_25.mtx"))
counts = csr_matrix(counts.transpose()) #transposing to cell x gene matrix and converting to csr format
#Contains metadata, barcodes, and umap embeddings 
metadata = pd.read_csv(os.path.join(ConversionDirectory, "HHA_SCRNA_Multiome_Melano_FullConversion_Metadata_11_18_25.csv"))
#Contains gene names 
gene_names = pd.read_csv(os.path.join(ConversionDirectory, "HHA_SCRNA_Multiome_Melano_FullConversion_GeneNames_11_18_25.csv"))
#Converting to anndata object
Melano = ad.AnnData(counts) #creating anndata object
Melano.obs_names = metadata["barcode"] #Adding barcodes as obs names 
Melano.var_names = gene_names["gene"] #adding gene names 
print(Melano)
print(Melano.obs_names[:10])
print(Melano.var_names[:10])
scvi = pd.read_csv(os.path.join(ConversionDirectory, "HHA_SCRNA_Multiome_Melano_FullConversion_scVI_Embeddings_11_18_25.csv"))
scvi["barcode"] = metadata["barcode"]
#------Adding metadata to anndata--------#
Melano.obs = metadata
Melano.obs_names = metadata["barcode"]

#-----Adding dimensionality reductions to obsm-----#
X_umap = metadata.set_index("barcode")[["UMAP_1", "UMAP_2"]]
Melano.obsm['X_umap'] = X_umap.loc[Melano.obs_names].values
X_scvi = scvi.set_index("barcode")
Melano.obsm['X_scvi'] = X_scvi.loc[Melano.obs_names].values
print(Melano.obs.head())

In [ ]:
#-----Plotting Clusters and Genes------#
#Dictionary mapping colors to each cluster
color_dict = {"Melanocytes_I": "#606372",
             "Melanocytes_II": "#92A185",
             "Melanocytes_III": "#DEC18C"}
#Clusters
sc.pl.umap(
    Melano,
    color="MelanoAnnotation",
    frameon=False,
    add_outline=False,
    palette = color_dict,
    size = 15)

In [ ]:
#-----Crossmapping var_names to EnsemblID--------#
Melano.var["gene"] = Melano.var_names
symbol_to_ensembl = dict(zip(HHA.var["names"], HHA.var["ensembl"]))
def map_to_ensembl(name):
    if name.startswith("ENSG"):
        return name
    return symbol_to_ensembl.get(name, "Unknown")
Melano.var["ensembl"] = Melano.var["gene"].apply(map_to_ensembl)

#-------Creating EnsemblID column and set as index--------#
Melano = Melano[:, Melano.var["ensembl"] != "Unknown"].copy()
new_index = Melano.var["ensembl"]
Melano.var.index = new_index
Melano.var_names = new_index
Melano.var

In [ ]:
#--------------Saving Melanocyte Object------------#
del(Melano.obs['Likely_Ruptured_075'])
Melano.write_h5ad("/projects/b1217/HHA/Melanocyte_Plots/Melano_AnnData/Melano_11_21_25.h5ad")

In [ ]:
#-------Second Iteration TACCO Run on spots mapped to IFE-------#
Melano_Pops = ["Melanocytes I", "Melanocytes II"]
#Subsetting for spots mapped to a IFE cell type
b2c_Melano = b2c_adata[b2c_adata.obs["FinalAnnotation"].isin(Melano_Pops)].copy()
b2c_Melano

In [ ]:
#--------Plotting Subset-------#
fig, ax = plt.subplots(figsize=(20, 20))
sc.pl.spatial(b2c_Melano, color=["FinalAnnotation"], img_key="hires", ax=ax)

In [ ]:
#------Rerunning Annotation using Matrix Anndata Object------#
tc.tl.annotate(b2c_Melano, Melano, annotation_key='MelanoAnnotation', result_key='MelanoAnnotation')

In [ ]:
#-----Retrieving Best Annotation Mapping for Each Cluster------#
tc.utils.get_maximum_annotation(b2c_Melano, 'MelanoAnnotation', result_key='MelanoAnnotation')

In [ ]:
#--------Plotting Final TACCO Annotations in Space------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(b2c_Melano, color=["MelanoAnnotation"], img_key="0.5_mpp_150_buffer", basis="spatial_cropped_150_buffer", ax=ax, palette = color_dict)
plt.tight_layout()

In [ ]:
#---------------Pulling Scores from Obsm----------------#
Annot_Scores = b2c_Melano.obsm['MelanoAnnotation'].copy()
Annot_Names = [x.replace(" ", "_") + "_Score" for x in Annot_Scores.columns]
Annot_Scores.columns = Annot_Names
Annot_Scores
#Adding to obs
for Score in Annot_Scores.columns:
    b2c_Melano.obs[Score] = Annot_Scores[Score]
b2c_Melano.obs

In [ ]:
#---------------Saving Object----------------#
b2c_Melano.write_h5ad("/projects/b1217/HHA/Bulb_Spatial/HHA_B2C_TACCO_Melano_11_21_25.h5ad")